In [2]:
import numpy as np
import math
import numpy.polynomial.polynomial as poly
import matplotlib.pyplot as plt

In [16]:
def decToRad(dec):
    return float(dec)/180*math.pi

def HMStoDeg(h:float,m:float,s:float,convert=False):
    try: h,m,s=float(h),float(m),float(s)
    except: raise Exception("Values must be floats")
    res = h*15 + m/60*15 + s/3600*15
    res%=360
    return np.deg2rad(res) if convert else res

def DMStoDeg(d:float,arcm:float,arcs:float,convert=False)->float:
    return np.deg2rad(math.copysign(abs(d)+arcm/60+arcs/3600,d)) if convert else math.copysign(abs(d)+arcm/60+arcs/3600,d)

def RAdecimalToHMS(dec:float)->tuple:
    dec%=360
    h=dec/15
    m=(dec%15)/15*60
    s=m%1*60
    h//=1
    m//=1
    #print((h,m,s))
    return (h,m,s)

def DECdecimalToDMS(dec:float)->tuple:
    d=math.trunc(dec)
    dec=abs(dec)-abs(d)
    m=math.trunc(dec*60)
    dec-=m/60
    s=dec*3600
    if (round(s,0)-s<0.00001):s=round(s,0)
    #print((d,m,s))
    return (d,m,s)

In [17]:
def formatInputInfo(file:str):
    info=np.loadtxt(file,dtype=str,delimiter=",")
    time=np.array([float(info[i,0]) for i in range(1,len(info))])
    timestamps=np.copy(info[1:,1])
    #print(timestamps)
    return time,np.array([([float(info[i][2]),float(info[i][3]),float(info[i][4])], 
                    [float(info[i][5]),float(info[i][6]),float(info[i][7])]) for i in range(1,len(info))]), timestamps

def formatResultInfo():
    info=np.loadtxt("/home/soonali/Documents/SoongResults.txt",dtype=str,delimiter=",")
    return np.array([[float(info[i][j]) for j in range(2,14)] for i in range(1,len(info))])

def cross(v1:list, v2:list)->list:
    '''Returns the cross product of two 3D vectors'''
    return [(v1[1]*v2[2] - v1[2]*v2[1]),-(v1[0]*v2[2] - v1[2]*v2[0]),(v1[0]*v2[1] - v1[1]*v2[0])]

def dot(v1:list,v2:list)->list:
    return sum([v1[i]*v2[i] for i in range(len(v1))])

def det2x2(m)->float:
    '''Returns the determinant of a 2x2'''
    return m[0][0]*m[1][1]-m[0][1]*m[1][0]

def angMoment(pos,vel):
    res=cross(pos,vel)
    return np.array([round(res[0]/(2*math.pi)*365.2568983,6), round(res[1]/(2*math.pi)*365.2568983,6), round(res[2]/(2*math.pi)*365.2568983,6)])

def getAngle(sin:float,cos:float)->float:
    '''Returns the arcsin of a value as an angle (in radians) in the correct quadrant'''
    return math.atan2(sin,cos) % (math.pi*2)

def getMag(vec):
    return math.sqrt(sum([vec[i]**2 for i in range(len(vec))]))
    
def newton(func,der,init,err): #function, derivative, inital_guess, error_tolerance
    prev=-1e10
    while abs(init-prev)>=err:
        prev,init=init,init-func(init)/der(init)
    return init

def rotZX(v,alpha, beta):
    '''rotates around z axis with alpha, then rotates around x axis with beta'''
    z=np.array([[np.cos(alpha),-np.sin(alpha),0],
                [np.sin(alpha),np.cos(alpha),0],
                [0,0,1]])
    x=np.array([[1,0,0],
                [0,np.cos(beta),-np.sin(beta)],
                [0,np.sin(beta),np.cos(beta)]])
    return np.dot(np.matmul(x,z),v)

def error(acc,exp):
    return math.copysign(abs(acc-exp)/acc*100,1)

In [22]:
class ODElements:
    '''Represents and calculates orbital elements'''
    
    def __init__(self, pos, vel, time):
        # constants
        self.k = 0.0172020989484 # au^(3/2)/day
        
        self.time=time
        #print(time)
        self.vel=vel
        self.vel=np.array([self.vel[i]/(2*math.pi)*365.2568983 for i in range(3)])
        self.pos=pos
        #print(vel,pos)
        self.angMoment=self.getAngMoment()
        
        self.mu=1
        self.a=self.getSemiMajor() # semi-major axis
        self.e= self.getEcc() # eccentricity
        self.b=None # semi-minor axis
        self.i=self.getInc() # inclination
        self.o=self.getLongAsc() # longitude of ascending node
        self.v=self.getTrueAnom() # true anomaly
        self.w=self.getArgPer() # argument of perihelion
        self.M=self.getMeanAnomaly() # mean anomaly
        self.T=self.getPeriT() # time of perihelion passage T
      
    def getInfo(self):
        '''Returns the info from line'''
        return self.info
    
    def getPos(self):
        return self.pos
    
    def getVel(self):
        return self.vel
    
    def getPosMag(self):
        return math.sqrt(sum([self.pos[i]**2 for i in range(3)]))
    
    def getVelMag(self):
        return math.sqrt(sum([self.vel[i]**2 for i in range(3)]))
    
    def getAngMoment(self):
        '''Returns the angular momentum'''
        pos,vel=self.getPos(),self.getVel()
        res=cross(pos,vel)
        return np.array([res[0], res[1], res[2]])
    
    def getAngMomentMag(self):
        return math.sqrt(sum([self.angMoment[i]**2 for i in range(3)]))
    
    def getSemiMajor(self):
        '''Uses vis-viva'''
        #print(self.getPosMag())
        #print(self.vel)
        #print(self.mu)
        return 1/(2/self.getPosMag() - dot(self.vel,self.vel)/self.mu)
    
    def getEcc(self):
        #print(self.a)
        return math.sqrt(1-dot(self.angMoment, self.angMoment)/(self.mu*self.a))
    
    def getInc(self):
        #print(self.angMoment)
        #print(self.getAngMomentMag())
        return math.acos(self.angMoment[2]/self.getAngMomentMag())*180/math.pi
    
    def getLongAsc(self):
        s=self.angMoment[0]/(self.getAngMomentMag()*math.sin(self.i/math.pi/180))
        c=-self.angMoment[1]/(self.getAngMomentMag()*math.sin(self.i/math.pi/180))
        return getAngle(s,c)*180/math.pi
    
    def getArgPer(self):
        x,y,z=self.pos[0],self.pos[1],self.pos[2]
        s=z/(self.getPosMag()*math.sin(self.i*math.pi/180))
        c=(x*math.cos(self.o*math.pi/180) + y*math.sin(self.o*math.pi/180))/self.getPosMag()
        U=getAngle(s,c)*180/math.pi
        return (U-self.v)%360
    
    def getTrueAnom(self):
        c=1/self.e*(self.a*(1-self.e**2)/self.getPosMag() - 1)
        s=self.a*(1-self.e**2)/(self.getAngMomentMag() * self.e)*dot(self.pos,self.vel)/self.getPosMag()
        return getAngle(s,c)*180/math.pi
    
    def getPeriT(self):
        n=self.k/(self.a**(3/2))
        return self.time-self.M*math.pi/180/n
    
    def getMeanAnomaly(self):
        s=(self.getPosMag()*math.sin(self.v*math.pi/180))/(self.a*math.sqrt(1-self.e**2))
        c=(self.e+math.cos(self.v*math.pi/180))/(1+self.e*math.cos(self.v*math.pi/180))
        E=getAngle(s,c)
        return (E-self.e*math.sin(E))*180/math.pi
    
    def printError(self, results):
        # EC, QR, IN, OM, W, Tp, N, MA, TA, A, AD, PR,

        print("Semi-major axis:", results[9], self.a, error(results[9],self.a))
        print("Eccentricity:", results[0], self.e, error(results[0],self.e))
        print("Inclination:",results[2],self.i, error(results[2],self.i))
        print("Longitude of Ascending Node:",results[3],self.o, error(results[3],self.o))
        print("True anomaly:",results[8],self.v,error(results[8],self.v))
        print("Argument of perihelion:",results[4],self.w,error(results[4],self.w))
        print("Time of Perihelion Passage T:",results[5],self.T,error(results[5],self.T))
        print("Mean Anomaly:",results[7],self.M,error(results[7],self.M))
        #print(od.a, od.e, od.i, od.o, od.v, od.w)
        
    def getElements(self):
        '''a,e,i,o,v,w,T,M'''
        return self.a, self.e, self.i, self.o, self.v, self.w, self.T, self.M


def testODElements():
    file = "/home/soonali/Documents/SoongInput2.txt"
    time = '2018-Jul-14 00:00:00'
    times,data,timestamps=formatInputInfo(file)

        
    line = 0
    if time!=0:
        for i in range(len(timestamps)):
            if time in timestamps[i]: 
                line = i
                #print(line)
                break
    
    results=formatResultInfo()[24]
    #print(formatResultInfo())
    time,info=times[line],data[line]

    od=ODElements(info[0],info[1],time)
    od.printError(results)
    
testODElements()


Semi-major axis: 1.056800055744403 1.0568000556596946 8.015561884787836e-09
Eccentricity: 0.3442331093278357 0.3442331094157307 2.5533576582325764e-08
Inclination: 25.1552516656358 25.155251665637707 7.584131023113933e-12
Longitude of Ascending Node: 236.2379806531768 236.23798065316336 5.690646577409404e-12
True anomaly: 158.9559248709949 158.9559248763771 3.385982069686107e-09
Argument of perihelion: 255.5046142809498 255.5046142714118 3.732998971925637e-09
Time of Perihelion Passage T: 2458158.720849546 2458158.720849547 3.7886999188303465e-14
Mean Anomaly: 140.4194576216498 140.41945762513234 2.4800982265579533e-09


In [31]:
class OD:
    
    def __init__(self,file,time=0):
        self.times,self.data, self.timestamps=formatInputInfo(file)
        
        line = 0
        if time!=0:
            for i in range(len(self.timestamps)):
                if time in self.timestamps[i]: 
                    line = i
                    break
        
        self.time,self.info=self.times[line],self.data[line]
        self.pos=self.info[0]
        self.vel=self.info[1]
        self.od=ODElements(self.info[0],self.info[1],self.time)
        self.a,self.e,self.i,self.o,self.v,self.w,self.T,self.M = self.od.getElements()
        
        # constants
        self.k = 0.0172020989484 #Gaussian gravitational constant  
        self.cAU = 173.144643267 #speed of light in au/(mean solar)day  
        self.mu = 1
        self.eps = np.radians(23.4374) #Earth's obliquity
        
    def getInfoAtTime(self, time:str, override=False):
        for i in range(len(self.timestamps)):
            if time in self.timestamps[i]: 
                line = i
                break
        if override:
            self.time,self.info=self.times[line],self.data[line]
            self.pos=self.info[0]
            self.vel=self.info[1]
            self.od=ODElements(self.info[0],self.info[1],self.time)
            self.a,self.e,self.i,self.o,self.v,self.w,self.T,self.M = self.od.getElements()
            return self.getElements()
        else:
            time,info=self.times[line],self.data[line]
            od=ODElements(info[0],info[1],time)
            return od.getElements()
    
    def getElements(self):
        '''a,e,i,o,v,w,T,M'''
        return self.a, self.e, self.i, self.o, self.v, self.w, self.T, self.M
    
    def getT(self):
        return self.T
    
    def SEL(self, taus, sunR2, rhohat2, coefD):
        '''Scalar Equation of Lagrange'''
        T1,T3,T=taus[0],taus[1],taus[2]
        D0,D21,D22,D23=coefD[0],coefD[1],coefD[2],coefD[3]
        A1=T3/T
        B1=A1/6*(T**2-T3**2)
        A3=-T1/T
        B3=A3/6*(T**2-T1**2)
        A=(A1*D21-D22+A3*D23)/(-D0)
        B=(B1*D21+B3*D23)/(-D0)
        
        E=-2*(dot(rhohat2, sunR2))
        F=dot(sunR2,sunR2)
        
        a=-(A**2+A*E+F)
        b=-self.od.mu*(2*A*B+B*E)
        c=-self.od.mu**2*B**2
        
        coef=[c,0,0,b,0,0,a,0,1]#[1,0,a,0,0,b,0,0,c]
        res=poly.polyroots(coef)

        roots=[]
        for val in res: 
            if np.isreal(val) and np.real(val)>0: roots.append(np.real(val))

        rhos=[A+B/roots[i]**3 for i in range(len(roots))]
        return roots,rhos
    
    def ephemeris(self, time:float, date:str):
        '''expect time to be in JD'''
        #self.getInfoAtTime(date)
        n=self.k*math.sqrt(self.mu/(self.a**3))
        M=n*(time-self.T)
        E=newton(lambda E:M - E + self.e*np.sin(E), lambda E: -1+self.e*np.cos(E), M, 1e-13)
        #print("n,M,t:",n,M,time)
        #print("E:",E)
        #print("T:",self.T)
        
        pos=np.array([self.a*math.cos(E)-self.a*self.e, self.a*math.sqrt(1-self.e**2)*math.sin(E), 0])
        #print("pos before rot:",pos)
        
        # the four rotations
        pos=rotZX(pos,self.w*np.pi/180,self.i*np.pi/180)
        pos=rotZX(pos,self.o*np.pi/180,self.eps)
        
        #print("pos:",pos)
        rho=pos+self.getSunPos(date)
        #print("sun pos:",self.getSunPos(date))
        #print("rho:",rho)
        rhohat=rho/getMag(rho)
        
        #print("rhohat:",rhohat)
        
        dec=math.asin(rhohat[2])
        cra=rhohat[0]/math.cos(dec)
        sra=rhohat[1]/math.cos(dec)

        ra=getAngle(sra,cra)
        
        dec*=180/math.pi
        ra*=180/math.pi
        
        dec=DECdecimalToDMS(dec)
        ra=RAdecimalToHMS(ra)
        
        #print("calculated:")
        #print("ra:", ra)
        #print("dec:",dec)
        
        return ra,dec
        
    def getSunPos(self, date:str):
        info=np.loadtxt("/home/soonali/Documents/SunPos.txt",dtype=str,delimiter=",")
        #print(info)
        timestamps=info[:,1]
        for i in range(len(timestamps)):
            if date in timestamps[i]: 
                line = i
                break
       
        stuff=info[line,:]
        x,y,z=float(stuff[2]),float(stuff[3]),float(stuff[4])
        return np.array([x,y,z])
    
    def getSunMag(self,date:str):
        '''magnitude of position from sun to earth'''
        pos=getSunPos(date)
        x,y,z=pos[0],pos[1],pos[2]
        return math.sqrt(x**2+y**2+z**2)
        # or just use getMag(getSunPos(date))
        
    def fg(self, tau:float, r2, r2dot, flag:bool):
        u=self.mu/getMag(r2)**3
        z=dot(r2,r2dot)/(dot(r2,r2))
        q=dot(r2dot,r2dot)/(dot(r2,r2))-u
        
        f=1-1/2*u*tau**2+1/2*u*z*tau**3
        g=tau-1/6*u*tau**3
        if flag:
            f+=1/24*(3*u*q-15*u*z**2+u**2)*tau**4
            g+=1/4*u*z*tau**4
        
        return f, g
        
    def getFGVals(self, tau1:float, tau3:float, r2, r2dot, flag1:bool, flag3:bool):
        f1,g1=self.fg(tau1,r2,r2dot,flag1)
        f3,g3=self.fg(tau3,r2,r2dot,flag3)
        return [f1,f3], [g1,g3]
    
    def getDCoef(self, ra, dec, R1, R2, R3):
        '''expects ra and dec to all be in radians already'''
        ra1,ra2,ra3=ra[0],ra[1],ra[2]
        dec1,dec2,dec3=dec[0],dec[1],dec[2]
        
        rhohat1=np.array([np.cos(ra1)*np.cos(dec1), np.sin(ra1)*np.cos(dec1), np.sin(dec1)])
        rhohat2=np.array([np.cos(ra2)*np.cos(dec2), np.sin(ra2)*np.cos(dec2), np.sin(dec2)])
        rhohat3=np.array([np.cos(ra3)*np.cos(dec3), np.sin(ra3)*np.cos(dec3), np.sin(dec3)])
        
        D0=dot(rhohat1, cross(rhohat2,rhohat3))
        D11=dot(cross(R1, rhohat2),rhohat3)
        D12=dot(cross(R2, rhohat2),rhohat3)
        D13=dot(cross(R3, rhohat2),rhohat3)
        
        D21=dot(cross(rhohat1,R1), rhohat3)
        D22=dot(cross(rhohat1,R2), rhohat3)
        D23=dot(cross(rhohat1,R3), rhohat3)
        
        D31=dot(rhohat1, cross(rhohat2,R1))
        D32=dot(rhohat1, cross(rhohat2,R2))
        D33=dot(rhohat1, cross(rhohat2,R3))
     
        return [D0,D11,D12,D13,D21,D22,D23,D31,D32,D33]
        
        

    

In [32]:
def testEphemeris():
    file = "/home/soonali/Documents/SoongInput2.txt"
    
    date = '2018-Jul-14 00:00:00'
    jdtime = 2458313.5000000
    od=OD(file, date)

    newdate = '2018-Aug-03 00:00:00'
    newjdtime = 2458333.5000000
 
    #print(od.getElements())
    #print(od.getInfoAtTime(newdate,override=True))
    
    ra,dec=od.ephemeris(newjdtime, newdate)
    
    # for jul 14 00:00:00
    #ra,dec=od.ephemeris(jdtime, date)
    #print("\n% error ra:", error(HMStoDeg(18,14,30.53),HMStoDeg(ra[0],ra[1],ra[2])))
    #print("% error dec:", error(DMStoDeg(36,9,46.0),DMStoDeg(dec[0],dec[1],dec[2])))

    #print("\nexpected:")
    #print("ra: 18 14 30.53")
    #print("dec: +36 09 46.0")
    
    print("calculated:")
    print("ra:", ra)
    print("dec:",dec)
    
    print("\n% error ra:", error(HMStoDeg(17,42,21.12),HMStoDeg(ra[0],ra[1],ra[2])))
    print("% error dec:", error(DMStoDeg(31,52,26.8),DMStoDeg(dec[0],dec[1],dec[2])))

    print("\nexpected:")
    print("ra: 17 42 21.12")
    print("dec: +31 52 26.8")
    
testEphemeris()

calculated:
ra: (17.0, 42.0, 22.10681952482446)
dec: (31, 52, 46.0)

% error ra: 0.001548167846468828
% error dec: 0.016732492757966136

expected:
ra: 17 42 21.12
dec: +31 52 26.8


In [25]:
def testFG():
    tau1 = -0.32618617484601165  
    tau3 = 0.0508408854033231  
    r2 = [0.26799552002875776, -1.3726277901924608, -0.5026729612047128]  
    r2dot = [0.8456809141954584, -0.3838382184712308, 0.14215854191172816] 
    
    file = "/home/soonali/Documents/SoongInput2.txt"
    date = '2018-Jul-14 00:00:00'
    jdtime = 2458313.5000000
    od=OD(file, date)
    
    f,g=od.getFGVals(tau1,tau3,r2,r2dot,False,False)
    f1,f3=f[0],f[1]
    g1,g3=g[0],g[1]
    
    print("Test case 2:")
    print("f1:",f1,error(0.9821596284506794,f1))
    print("f3:",f3,error(0.9996124342607986,f3))
    print("g1:",g1,error(-0.32442392608030396,g1))
    print("g3:",g3,error(0.05083421257607972,g3))
    
    tau1 = -0.3261857571141891  
    tau3 = 0.05084081855693949  
    r2 = [0.26662393644794813, -1.381475976476564, -0.5048589337503169]  
    r2dot = [0.8442117090940343, -0.39728396707075087, 0.14202728258915864]  
    
    f,g=od.getFGVals(tau1,tau3,r2,r2dot,True,True)
    f1,f3=f[0],f[1]
    g1,g3=g[0],g[1]
    
    print("\nTest case 3:")
    print("f1:",f1,error(0.9823149707782799  ,f1))
    print("f3:",f3,error(0.99961917185922  ,f3))
    print("g1:",g1,error(-0.32418770657924106  ,g1))
    print("g3:",g3,error(0.05083441832100904,g3))
    
    
testFG()

Test case 2:
f1: 0.9821596284506794 0.0
f3: 0.9996124342607986 0.0
g1: -0.32442392608030396 0.0
g3: 0.05083421257607972 0.0

Test case 3:
f1: 0.9823149707782799 0.0
f3: 0.99961917185922 0.0
g1: -0.32418770657924106 0.0
g3: 0.05083441832100904 0.0


In [26]:
def testDCoef():
    """2021 06 25 00:00:00.000 18:25:08.44 -17:26:41.3 -5.985728598861461E-02 9.309676159065817E-01 4.035414693476737E-01 
 
    2021 07 05 00:00:00.000 18:15:28.85 -16:27:16.5 -2.271502585826002E-01 9.092709064712199E-01 3.941342306093848E-01 
 
    2021 07 15 00:00:00.000 18:05:40.89 -15:30:48.9 -3.881336047533506E-01 8.619617590425438E-01 3.736284118981542E-01"""
    
    ra=[HMStoDeg(18,25,8.44, convert=True),HMStoDeg(18,15,28.85, convert=True),HMStoDeg(18,5,40.89, convert=True)]
    dec=[DMStoDeg(-17,26,41.3,convert=True), DMStoDeg(-16,27,16.5,convert=True), DMStoDeg(-15,30,48.9,convert=True)]
    R1=[-5.985728598861461E-02,9.309676159065817E-01,4.035414693476737E-01]
    R2=[-2.271502585826002E-01,9.092709064712199E-01,3.941342306093848E-01]
    R3=[-3.881336047533506E-01,8.619617590425438E-01,3.736284118981542E-01]
    
    date = '2018-Jul-14 00:00:00'
    file = "/home/soonali/Documents/SoongInput2.txt"
    od=OD(file, date)
    dCoefs=od.getDCoef(ra,dec,R1,R2,R3)
    print(dCoefs)
    
    ans=[7.153072967571068e-05,-0.0051069590315919065,-0.0021329782464551625,
     0.000902451104005289,0.010014747188866925, 0.0038526558958926135, -0.0024208093416812626, -0.004989761292019754, 
     -0.001794759627912354, 0.0014522846235393017]
    
    for i in range(10):
        print("error:", error(dCoefs[i],ans[i]))
    
    
testDCoef()

[7.15307296756968e-05, -0.005106959031591908, -0.0021329782464551555, 0.000902451104005289, 0.01001474718886704, 0.0038526558958927246, -0.0024208093416811793, -0.0049897612920198585, -0.0017947596279124511, 0.0014522846235392184]
error: 1.9401155098980568e-11
error: 3.396783614761191e-14
error: 3.253147994096579e-13
error: 0.0
error: 1.1432315489875251e-12
error: 2.881708241342689e-12
error: 3.4396234933999593e-12
error: 2.085939636532701e-12
error: 5.412675499486995e-12
error: 5.7334991707042714e-12
